In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../data/raw/volcan_data_2010.csv')

In [4]:
df.head()

,Category,Description,Nutrient Data Bank Number,Alpha Carotene,Beta Carotene,Beta Cryptoxanthin,Carbohydrate,Cholesterol,Choline,Fiber,...,Phosphorus,Potassium,Sodium,Zinc,Vitamin A - RAE,Vitamin B12,Vitamin B6,Vitamin C,Vitamin E,Vitamin K
0,Milk,"Milk, human",11000000,0,7,0,6.89,14,16.0,0.0,...,14,51,17,0.17,61,0.05,0.011,5.0,0.08,0.3
1,Milk,"Milk, NFS",11100000,0,4,0,4.87,8,17.9,0.0,...,103,157,39,0.42,59,0.56,0.060,0.1,0.03,0.2
2,Milk,"Milk, whole",11111000,0,7,0,4.67,12,17.8,0.0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
3,Milk,"Milk, low sodium, whole",11111100,0,7,0,4.46,14,16.0,0.0,...,86,253,3,0.38,29,0.36,0.034,0.9,0.08,0.3
4,Milk,"Milk, calcium fortified, whole",11111150,0,7,0,4.67,12,17.8,0.0,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3


In [5]:
df.shape

(7083, 38)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7083 entries, 0 to 7082
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Category                   7083 non-null   object 
 1   Description                7083 non-null   object 
 2   Nutrient Data Bank Number  7083 non-null   int64  
 3   Alpha Carotene             7083 non-null   int64  
 4   Beta Carotene              7083 non-null   int64  
 5   Beta Cryptoxanthin         7083 non-null   int64  
 6   Carbohydrate               7083 non-null   float64
 7   Cholesterol                7083 non-null   int64  
 8   Choline                    7083 non-null   float64
 9   Fiber                      7083 non-null   float64
 10  Lutein and Zeaxanthin      7083 non-null   int64  
 11  Lycopene                   7083 non-null   int64  
 12  Niacin                     7083 non-null   float64
 13  Protein                    7083 non-null   float

In [7]:
df.describe(include='all')

,Category,Description,Nutrient Data Bank Number,Alpha Carotene,Beta Carotene,Beta Cryptoxanthin,Carbohydrate,Cholesterol,Choline,Fiber,...,Phosphorus,Potassium,Sodium,Zinc,Vitamin A - RAE,Vitamin B12,Vitamin B6,Vitamin C,Vitamin E,Vitamin K
count,7083,7083,7.083000e+03,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,...,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000,7083.000000
unique,2429,7083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Infant formula,"Milk, human",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,173,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,4.884965e+07,43.759283,255.363405,4.862346,20.828039,34.458845,34.440802,1.703840,...,133.088945,217.127488,340.503459,1.281451,73.139065,0.705177,0.197528,5.696216,1.086955,14.206607
std,NaN,NaN,2.425159e+07,259.520139,995.709951,38.047657,21.887363,77.479105,43.901497,2.558168,...,126.498700,204.812523,350.333632,2.917233,241.913126,2.502821,0.374608,15.548490,3.133890,54.944096
min,NaN,NaN,1.100000e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,2.715077e+07,0.000000,0.000000,0.000000,5.650000,0.000000,10.000000,0.100000,...,48.000000,111.000000,124.000000,0.360000,2.000000,0.000000,0.052000,0.000000,0.230000,0.800000
50%,NaN,NaN,5.326020e+07,0.000000,8.000000,0.000000,13.300000,8.000000,19.600000,1.000000,...,102.000000,183.000000,313.000000,0.680000,20.000000,0.180000,0.110000,0.700000,0.550000,3.800000
75%,NaN,NaN,6.720345e+07,1.000000,73.000000,1.000000,26.195000,46.000000,44.200000,2.100000,...,189.000000,270.500000,454.000000,1.360000,61.000000,0.550000,0.203000,5.300000,1.110000,9.300000


In [8]:
df_cleaned = df.copy()